<a name="top"></a><img src="source/SpinalHDL.png" alt="SpinalHDL based on Scala" style="width:320px;" />

Before running Spinal HDL code, be sure to load SpinalHDL Libraries  
**Note** : This may be a little slow when the first time load, please wait a moment to download Lib from remote.)   

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-spinal.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

##  Bit width inference usage restriction 
Example1:
```scala
  val c = UInt()
  c.getWidth   // X Failed !
  c:= a + b
  c.getWidth   // is ok, because c width already inferred by "c := a + b"
```
Example2:(setAll need getWidth) 
```scala
 val c = out UInt() 
 when(clc){c.setAll }.otherwise(c := a * b)  //failed
 when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
```

In [ ]:
class T2  extends Component{
    val a,b = in UInt(8 bits)
    val clc= in Bool()
    val c = out UInt() 
//  when(clc){c.setAll }.otherwise(c := a * b)  //failed   
  when(clc){c := a * b }.otherwise(c.setAll)  //It's OK
}
showRtl(new T2)

##  asBool issue
Please note that for 1 bit data operation, the first method is different from the latter two methods.
- a.asBits  
- a.lsb     
- a(0) 
```scala
val a= Bits(1 bits)
a.asBits //return new Bool，then tmp := a.lsb
a.lsb    //direct operate a(0) bit
a(0)     //direct operate a(0) bit
```

In [ ]:
class T1  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a.asBool.set()} //generate verilog beyond your expectations
}
showRtl(new T1)  

In [ ]:
class T2  extends Component{  
  val sel = in Bool()
  val a = Reg(Bits(1 bits)) init 0 
  when(sel){a(0).set()} 
  // when(sel){a.lsb.set()}  //also Ok
}
showRtl(new T2) 

### 4. Overlap or Latch?

In [ ]:
import scala.collection.mutable.ListBuffer
class T2 extends Component{
  val a =  in UInt(2 bits)
  val Lut = ListBuffer.fill(8)(UInt(8 bits))
  (0 to 7).foreach{ i =>      
    Lut(i) := i        
  }
  switch(a){
    is(0){
        Lut(0) := 3
        Lut(2) := 4
    }
    is(1){
        Lut(3) := 3
        Lut(4) := 4
    }
  is(2){
        Lut(3) := 3
        Lut(4) := 4
    } 
   default{
    (0 to 7).foreach{ i =>           
      Lut(i) := i        
    }
   }
  }
}
showRtl(new T2)      

##  val def in Trait
**Attation** val cause Error 

In [ ]:
trait PRNBase {
  val size: Int 

  val Mask = (1 << size) - 1    //attation 
  val Msb  = (1 << (size - 1))  //attation
}

object GPS extends PRNBase{
    val size = 1023
}

object BD extends PRNBase{
    val size = 2046
}
BD.Mask toHexString // return 0

In [ ]:
trait PRNBase {
  val size: Int 

  def Mask = (1 << size) - 1
  def Msb  = (1 << (size - 1))
}
object BD extends PRNBase{
    val size = 11
}
BD.Mask toHexString

##   Scala double definition  
```scala 
class TestDoubleDef{
  def foo(p:List[String]) = {}
  def foo(p:List[Int]) = {}
}
```
**raise Error:**
```sh
[error] double definition:
[error] method foo:(List[String])Unit and
[error] method foo:(List[Int])Unit at line 120
[error] have same type after erasure: (List)Unit 
```
Solution: https://stackoverflow.com/questions/3307427/scala-double-definition-2-methods-have-the-same-type-erasure/3544060#3544060

In [ ]:
object MyTransform{
  def apply(x: Int ): Double              = x + 0.00 
  def apply(x: List[Int] ): List[Double]  = x.map(_+0.00)
  def apply(x: List[Double] ): List[Double] = x.map(_+0.00)
}

**Use implict transform**

In [ ]:
case class IntList(list: List[Int])
case class DoubleList(list: List[Double])

implicit def Il(list: List[Int]) = IntList(list)
implicit def Dl(list: List[Double]) = DoubleList(list)

object FixTo{
  def apply(x: Int ): Double              = x + 0.00 
  def apply(x: IntList ): List[Double]    = x.list.map(_+0.00)
  def apply(x: DoubleList ): List[Double] = x.list.map(_+0.00)
}

In [ ]:
val a = FixTo(3)
val b = FixTo(DoubleList(List(1,2,3,4,5)))
val c = FixTo(List(1,2,3,4,5))

### Watch Dog


In [ ]:
class T2  extends Component{ 
    val a = out Bits(9 bit)
    a := 133
    class Dog{}
    val xiaogou = new Dog

}
showRtl(new T2) 

In [ ]:
class T2  extends Component{ 
   class Dog{
      def genTimer(n: Int) = {
         val timer = Reg(UInt(n bits)) init 0 
         val clearTimer = in Bool()
         when(clearTimer){
            timer init 0
         }.otherwise {
            timer := timer + 1
         }
         (clearTimer,timer)
      }
   }
   val xiaogou = new Dog
   val (weigou,timer) = xiaogou.genTimer(8)
} 
showRtl(new T2) 

In [ ]:
 class Dog {
      def genTimer(n: Int) = {
         val timer = Reg(UInt(n bits)) init 0 
         val clearTimer = in Bool()
         when(clearTimer){
            timer init 0
         }.otherwise {
            timer := timer + 1
         }
         (clearTimer,timer)
      }
   }

why not?

In [ ]:
class T2  extends Component{ 
   val xiaogou =  new Dog
   val (weigou,timer) = xiaogou.genTimer(8)
} 
showRtl(new T2) 

In [ ]:
class T2  extends Component{  
   val (weigou,timer) = (new Dog).genTimer(8)
} 
showRtl(new T2) 

### pretty print

In [ ]:
implicit class UtilsExpand(x: HertzNumber) {
      def toString0: String = {
      x.toBigDecimal match {
        case y if y > BigDecimal(1e12) => (y/BigDecimal(1e12)).toDouble + " THz"
        case y if y > BigDecimal(1e9)  => (y/BigDecimal(1e9)).toDouble + " GHz"
        case y if y > BigDecimal(1e6)  => (y/BigDecimal(1e6)).toDouble + " MHz"
        case y if y > BigDecimal(1e3)  => (y/BigDecimal(1e3)).toDouble + " KHz"
        case _ => x.toBigDecimal + "Hz"
      }
    }
  }
println((100.21 MHz).toString0)

In [ ]:
println(HertzNumber(800010).toString0)

In [ ]:
(2 KiB)/7

In [ ]:
2048/7.0

In [ ]:
implicit class ByteExpand(x: BigInt) {
    def pretty: String = {
      x match {
        case y if y >= (BigInt(1) << 80) => (y/(BigInt(1)<<80)) + " YiB"
        case y if y >= (BigInt(1) << 70) => (y/(BigInt(1)<<70)) + " ZiB"
        case y if y >= (BigInt(1) << 60) => (y/(BigInt(1)<<60)) + " EiB"
        case y if y >= (BigInt(1) << 50) => (y/(BigInt(1)<<50)) + " PiB"
        case y if y >= (BigInt(1) << 40) => (y/(BigInt(1)<<40)) + " TiB"
        case y if y >= (BigInt(1) << 30) => (y/(BigInt(1)<<30)) + " GiB"
        case y if y >= (BigInt(1) << 20) => (y/(BigInt(1)<<20)) + " MiB"
        case y if y >= (BigInt(1) << 10) => (y/(BigInt(1)<<10)) + " KiB"
        case _ => x + "Byte"
      }
    }
  }

println((BigInt(1) << 80 Byte).pretty)

In [ ]:
20 GiB

In [ ]:
BigDecimal(333.001) toBigInt